In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm

In [18]:
def filter1(row):
    if row["consumption"] > 2:
        return 1
    else:
        return 0

In [19]:
def filter2(row, Tlow):
    if row["consumption"] > Tlow:
        return row["consumption"]
    else:
        return 0

In [20]:
def duration(data, k):
    cur_length = 1
    while k + 1 <= len(data) - 1 and data["consumption_th"][k + 1]:
        cur_length += 1
        k += 1

    return cur_length, k + 1

In [21]:
#######################################################################################

In [22]:
#######################################################################################

In [23]:
#######################################################################################

In [24]:
def category1(row, Rate_EST):
    if row["consumption"] > max(
        Rate_EST + row["base_load"], row["regular_profile"] + Rate_EST * 1 / 2
    ):
        return 1
    elif row["consumption"] < Rate_EST:
        return -1000
    else:
        return 0


def category2(row, Rate_EST):
    threshold1 = Rate_EST * 2 / 3
    threshold2 = Rate_EST * 2 / 3
    if 5 <= row["Month"] <= 10:
        if row["dif1"] > threshold1 or row["dif2"] > threshold1:
            return 1
        else:
            return 0
    else:
        if row["dif1"] > threshold2 or row["dif2"] > threshold2:
            return 1
        else:
            return 0


def category3(row, Rate_EST):
    threshold1 = Rate_EST * 2 / 3
    threshold2 = Rate_EST * 2 / 3
    if 5 <= row["Month"] <= 10:
        if row["dif1"] < -threshold1 or row["dif2"] < -threshold1:
            return -1
        else:
            return 0
    else:
        if row["dif1"] < -threshold2 or row["dif2"] < -threshold2:
            return -1
        else:
            return 0


def ChargingDetection(start, end, mean, std, Rate_EST, dic):
    cnt = 1

    if (
        not data.iloc[start]["forward"] * data.iloc[start]["backward"]
        or (end - start) >= 24
    ):
        return False

    if start - end >= 3:
        threshold = 0.01 / (end - start)
    else:
        threshold = 0.01 / (end - start)

    for k in range(start, end):
        ###
        if not data.iloc[k]["forward"] * data.iloc[k]["backward"]:
            return False
        ###

        normal_load = data.iloc[k]["consumption"] - Rate_EST
        if (
            (norm.cdf(x=normal_load, loc=mean, scale=std) < threshold)
            or normal_load < data.iloc[k]["base_load"]
        ) and start <= k < end - 1:
            return False

        if (
            (
                norm.cdf(
                    x=data.iloc[k]["consumption"] - Rate_EST / 2, loc=mean, scale=std
                )
                < threshold
            )
            or data.iloc[k]["consumption"] - Rate_EST / 2 < data.iloc[k]["base_load"]
        ) and k >= end - 1:
            return False

        if data.iloc[k]["higher"] == 1:
            cnt += 1
        elif data.iloc[k]["higher"] < 0 and k < end - 1:
            cnt += data.iloc[k]["higher"]
        else:
            cnt -= 2
        if cnt <= 0:
            return False

    return True


def estimation(Rate_EST, n, Charging_period, bound):
    estimate = []
    for i in range(n):
        if i in Charging_period:
            estimate.append(Rate_EST)
        elif i in bound:
            estimate.append(Rate_EST / 2)
        else:
            estimate.append(0)
    return estimate


def ROC(data, P, N, n):
    TP, TN, FP, FN = 0, 0, 0, 0
    for i in range(n):
        car = data.iloc[i]["car"]
        grid = data.iloc[i]["grid"]
        solar = data.iloc[i]["solar"]
        estimation = data.iloc[i]["estimation"]
        if car > 0.5 and grid + solar >= car and estimation > 0.5:
            TP += 1
        elif (car < 0.5 or grid + solar < car) and estimation < 0.5:
            TN += 1
        elif (car < 0.5 or grid + solar < car) and estimation > 0.5:
            FP += 1
        elif car > 0.5 and grid + solar >= car and estimation < 0.5:
            FN += 1
    return TP / P, TN / N, FP / N, FN / P, (TP + TN) / n


def MSE(data):
    gap = sum([(i - j) ** 2 for i, j in zip(data["car"], data["estimation"])])
    return np.sqrt(gap / len(data))

In [25]:
def filter(candidate):
    delete = set()
    for i in range(len(candidate) - 2):
        if (
            candidate[i][1] + 2 == candidate[i + 1][0]
            or candidate[i][1] + 1 == candidate[i + 1][0]
        ):
            delete.add(i)
            delete.add(i + 1)
    candidate_filter = []
    for i in range(len(candidate)):
        if i not in delete:
            candidate_filter.append(candidate[i])

    return candidate_filter

In [26]:
#############################################################################

In [27]:
def ACfiltering(data):
    data["Tlow"] = data.apply(lambda x: filter1(x), axis=1)

    data_sub = data[data["Tlow"] == 1]
    Tlow = max(2.5, 1 / (2 * len(data_sub)) * sum(data_sub["consumption"]))

    data["consumption_th"] = data.apply(lambda x: filter2(x, Tlow), axis=1)

    visited = set()
    cur_position = 0
    T_seed = 15
    while cur_position <= len(data) - 1:
        if not data["consumption_th"][cur_position]:
            cur_position += 1
            continue

        cur_length, cur_position = duration(data, cur_position)
        if cur_length == 1:
            visited.add(cur_position - 1)
            break
        cur_position += 1

    prev_length, prev_position = cur_length, cur_position

    while cur_position <= len(data) - 1:
        if not data["consumption_th"][cur_position]:
            cur_position += 1
            continue

        cur_length, cur_position = duration(data, cur_position)
        if (
            cur_length <= 2.2 * prev_length
            and cur_position - cur_length - prev_position <= 3 * prev_length
            and cur_length <= 6
        ):
            for k in range(cur_position - cur_length, cur_position):
                visited.add(k)
            prev_length, prev_position = cur_length, cur_position
        else:
            while cur_position <= len(data) - 1:
                if not data["consumption_th"][cur_position]:
                    cur_position += 1
                    continue

                cur_length, cur_position = duration(data, cur_position)
                if cur_length == 1:
                    visited.add(cur_position - 1)
                    break
                cur_position += 1
            prev_length, prev_position = cur_length, cur_position

    forward = [1] * len(data)
    for k in visited:
        forward[k] = 0
    ############################
    data["forward"] = forward
    data = data.iloc[::-1]
    data.index = range(len(data))
    data["consumption_th"] = data.apply(lambda x: filter2(x, Tlow), axis=1)
    ###########################

    visited = set()
    cur_position = 0
    T_seed = 15
    while cur_position <= len(data) - 1:
        if not data["consumption_th"][cur_position]:
            cur_position += 1
            continue

        cur_length, cur_position = duration(data, cur_position)
        if cur_length == 1:
            visited.add(cur_position - 1)
            break
        cur_position += 1

    prev_length, prev_position = cur_length, cur_position

    while cur_position <= len(data) - 1:
        if not data["consumption_th"][cur_position]:
            cur_position += 1
            continue

        cur_length, cur_position = duration(data, cur_position)
        if (
            cur_length <= 2.2 * prev_length
            and cur_position - cur_length - prev_position <= 3 * prev_length
            and cur_length <= 6
        ):
            for k in range(cur_position - cur_length, cur_position):
                visited.add(k)
            prev_length, prev_position = cur_length, cur_position
        else:
            while cur_position <= len(data) - 1:
                if not data["consumption_th"][cur_position]:
                    cur_position += 1
                    continue

                cur_length, cur_position = duration(data, cur_position)
                if cur_length == 1:
                    visited.add(cur_position - 1)
                    break
                cur_position += 1
            prev_length, prev_position = cur_length, cur_position

    backward = [1] * len(data)
    for k in visited:
        backward[k] = 0

    data["backward"] = backward
    data = data.iloc[::-1]
    data.index = range(len(data))

    data.to_csv(
        "E:/Research/EV_load_disaggregation/data/Improved/forward_{0}.csv".format(
            customer_id
        ),
        index=False,
    )
    return data


def helper(data, start, end, visited):
    normal = []
    cur = start - 1
    while len(normal) <= 7:
        if cur - 1 not in visited:
            normal.append(data.iloc[cur - 1]["consumption"])
            cur -= 1
        else:
            cur -= 1
    cur = end + 1
    while len(normal) <= 15:
        if cur + 1 not in visited:
            normal.append(data.iloc[cur + 1]["consumption"])
            cur += 1
        else:
            cur += 1
    return np.mean(normal), np.std(normal)

In [28]:
lo, up = 25, 75

In [29]:
for customer_id in [4373, 1642, 8156, 6139, 7719]:
    Rate_EST = 3

    alpha = 0.1
    percent = 0.1

    data = pd.read_csv(
        "E:/Research/EV_load_disaggregation/data/Improved/EVuser_{0}.csv".format(
            customer_id
        )
    )
    data = ACfiltering(data)

    data_non = pd.read_csv(
        "E:/Research/EV_load_disaggregation/data/NonEV_User/NonEVuser_2335.csv"
    )
    profile = data_non[["Month", "Day", "Hour", "Minute", "consumption"]]
    for iid in [9278, 9019, 3039, 7800, 9160, 2818, 4031, 3456, 3538, 7536]:
        data_non = pd.read_csv(
            "E:/Research/EV_load_disaggregation/data/NonEV_User/NonEVuser_{0}.csv".format(
                iid
            )
        )
        data_non = data_non[["Month", "Day", "Hour", "Minute", "consumption"]]
        profile = pd.merge(profile, data_non, on=["Month", "Day", "Hour", "Minute"])
    profile["consumption"] = np.mean(profile.iloc[:, 4:], axis=1)
    profile = profile[["Month", "Day", "Hour", "Minute", "consumption"]]
    Non_EVProfile = profile.groupby(["Month", "Hour", "Minute"], as_index=False).mean()
    Non_EVProfile.rename(columns={"consumption": "consumption_NonEV"}, inplace=True)

    data_base = data[["Month", "consumption"]]
    base_load = data_base.groupby(["Month"], as_index=False).min()
    base_load.rename(columns={"consumption": "base_load"}, inplace=True)
    data = pd.merge(data, base_load, on=["Month"])

    dic = dict()
    for i in range(len(base_load)):
        dic[(base_load.iloc[i]["Month"])] = base_load.iloc[i]["base_load"]
    ###
    data["under"] = data.apply(
        lambda x: 1
        if (
            x["consumption"] < x["base_load"] + Rate_EST
            or x["forward"] == 0
            or x["backward"] == 0
        )
        else 0,
        axis=1,
    )
    data_NEV = data[data["under"] == 1]
    ###
    data_group = data_NEV.groupby(by=["Month", "Hour", "Minute"], as_index=False).mean()

    ratio = sum(Non_EVProfile["consumption_NonEV"]) / sum(data_group["consumption"])
    Non_EVProfile["consumption_NonEV"] = Non_EVProfile["consumption_NonEV"] / ratio

    Non_EVProfile = Non_EVProfile[["Month", "Hour", "Minute", "consumption_NonEV"]]

    data_group = pd.merge(
        data_group, Non_EVProfile, how="right", on=["Month", "Hour", "Minute"]
    )
    data_group = data_group.fillna(0)

    regular_profile = []

    for i in range(len(data_group)):
        if not data_group.iloc[i, :]["consumption"]:
            regular_profile.append(data_group.iloc[i, :]["consumption_NonEV"])
        else:
            regular_profile.append(
                (1 - alpha) * data_group.iloc[i, :]["consumption"]
                + alpha * data_group.iloc[i, :]["consumption_NonEV"]
            )

    data_group["regular_profile"] = regular_profile

    data_group = data_group[["Month", "Hour", "Minute", "regular_profile"]]
    data_total = pd.merge(data, data_group, on=["Month", "Hour", "Minute"])
    data_total = data_total.sort_values(by=["Month", "Day", "Hour", "Minute"])
    data_total["prev1"] = data_total["consumption"].shift(1)
    data_total["dif1"] = data_total["consumption"] - data_total["prev1"]
    data_total["prev2"] = data_total["consumption"].shift(2)
    data_total["dif2"] = data_total["consumption"] - data_total["prev2"]
    data_total.to_csv(
        "E:/Research/EV_load_disaggregation/data/Improved/EVuser_{0}_profile.csv".format(
            customer_id
        ),
        index=False,
    )

    epochs = 1
    for i in range(epochs):
        data = pd.read_csv(
            "E:/Research/EV_load_disaggregation/data/Improved/EVuser_{0}_profile.csv".format(
                customer_id
            )
        )

        ##################
        data = data.dropna()
        ##################

        data["higher"] = data.apply(lambda x: category1(x, Rate_EST), axis=1)
        data["start"] = data.apply(lambda x: category2(x, Rate_EST), axis=1)
        data["end"] = data.apply(lambda x: category3(x, Rate_EST), axis=1)
        start_candidate = []
        candidate = []

        for i in range(len(data)):
            if (
                data.iloc[i]["start"] == 1
                and data.iloc[i]["higher"] == 1
                and data.iloc[i]["forward"] * data.iloc[i]["backward"]
            ):
                start_candidate.append(i)

        visited = set()
        start_visited = set()
        for start in start_candidate:
            for end in range(start + 1, len(data)):
                if (
                    data.iloc[end]["end"] == -1 and data.iloc[end]["higher"] <= 0
                ) and start not in start_visited:
                    candidate.append([start, end])
                    start_visited.add(start)
                    for i in range(start, end + 1):
                        visited.add(i)
                    break
                elif end - start > 24:
                    break

        start_visited = set()
        for start in start_candidate:
            for end in range(start + 1, len(data)):
                if (
                    data.iloc[end]["end"] == -1
                    and data.iloc[end + 1]["end"] == -1
                    and data.iloc[end + 2]["end"] == -1
                    and start not in start_visited
                ):
                    candidate.append([start, end])
                    start_visited.add(start)
                    for i in range(start, end + 1):
                        visited.add(i)
                    break
                elif end - start > 24:
                    break

        candidate = filter(candidate)

        EVcharging = []
        for start, end in candidate:
            mean, std = helper(data, start, end, visited)
            if ChargingDetection(start, end, mean, std, Rate_EST, dic):
                EVcharging.append([start, end - 1])

        Chargingrate = []
        for start, end in EVcharging:
            Chargingrate.append(max(data.iloc[start]["dif2"], data.iloc[start]["dif1"]))

            #############################

        lower = np.percentile(Chargingrate, lo)
        upper = np.percentile(Chargingrate, up)
        Rate_EST = np.mean([i for i in Chargingrate if lower < i < upper])

        Charging_period = set()
        bound = set()
        for start, end in EVcharging:
            for k in range(start, end + 1):
                Charging_period.add(k)
            bound.add(start - 1)
            bound.add(end + 1)

        data["estimation"] = estimation(Rate_EST, len(data), Charging_period, bound)

        Non_EVPeriods = data[data["estimation"] < 1]
        Non_EVPeriods = Non_EVPeriods[["Month", "Day", "Hour", "Minute", "consumption"]]

        Non_EVPeriods.to_csv(
            "E:/Research/EV_load_disaggregation/data/Improved/EVuser_{0}_Updated.csv".format(
                customer_id
            ),
            index=False,
        )
        data.to_csv(
            "E:/Research/EV_load_disaggregation/data/Improved/EVuser_{0}_Estimation_Month.csv".format(
                customer_id
            ),
            index=False,
        )

        P = len([i for i in data["car"] if i > 0.5])
        N = len([i for i in data["car"] if i < 0.5])
        print(customer_id, Rate_EST)
        ######################UPDATE#####################

        Non_EVPeriods = pd.read_csv(
            "E:/Research/EV_load_disaggregation/data/Improved/EVuser_{0}_Updated.csv".format(
                customer_id
            )
        )
        user = pd.read_csv(
            "E:/Research/EV_load_disaggregation/data/Improved/EVuser_{0}.csv".format(
                customer_id
            )
        )

        user = user[
            ["Month", "Day", "Hour", "Minute", "consumption", "car", "grid", "solar"]
        ]
        user = pd.merge(user, base_load, how="left", on=["Month"])

        data_group = Non_EVPeriods.groupby(
            by=["Month", "Hour", "Minute"], as_index=False
        ).mean()
        data_group.rename(columns={"consumption": "regular_profile"}, inplace=True)

        data_group = data_group[["Month", "Hour", "Minute", "regular_profile"]]

        data_group = pd.merge(
            user, data_group, how="left", on=["Month", "Hour", "Minute"]
        )
        data_group = pd.merge(
            data_group, Non_EVProfile, how="right", on=["Month", "Hour", "Minute"]
        )
        data_group = data_group.fillna(0)

        regular_profile = []

        for i in range(len(data_group)):
            if not data_group.iloc[i, :]["regular_profile"]:
                regular_profile.append(data_group.iloc[i, :]["consumption_NonEV"])
            else:
                regular_profile.append(
                    (1 - percent) * data_group.iloc[i, :]["regular_profile"]
                    + percent * data_group.iloc[i, :]["consumption_NonEV"]
                )
        data_group["regular_profile"] = regular_profile

        data_group = data_group.sort_values(by=["Month", "Day", "Hour", "Minute"])
        data_group["prev1"] = data_group["consumption"].shift(1)
        data_group["dif1"] = data_group["consumption"] - data_group["prev1"]
        data_group["prev2"] = data_group["consumption"].shift(2)
        data_group["dif2"] = data_group["consumption"] - data_group["prev2"]
        data_group = data_group.sort_values(by=["Month", "Day", "Hour", "Minute"])

        for_back = pd.read_csv(
            "E:/Research/EV_load_disaggregation/data/Improved/forward_{0}.csv".format(
                customer_id
            )
        )
        for_back = for_back[["Month", "Day", "Hour", "Minute", "forward", "backward"]]
        data_group = pd.merge(
            data_group, for_back, on=["Month", "Day", "Hour", "Minute"]
        )

        data_group.to_csv(
            "E:/Research/EV_load_disaggregation/data/Improved/EVuser_{0}_profile.csv".format(
                customer_id
            ),
            index=False,
        )

D:\Anaconda\Package\lib\site-packages\ipykernel_launcher.py:15: FutureWarning: Passing 'suffixes' which cause duplicate columns {'consumption_x'} in the result is deprecated and will raise a MergeError in a future version.
  from ipykernel import kernelapp as app


4373 3.288215163934426
1642 3.3721935483870964
8156 3.3186532258064516
6139 3.2633638059701493
7719 3.2106015625


In [30]:
np.percentile(data["temperature"], 80)

28.85

In [31]:
np.percentile(data["temperature"], 20)

11.0